In [18]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import math

In [61]:
def read_timeseries_data(subnum, dataloc="../tseries", runtype="pair", runnum=1):
    fname = f"{dataloc}/sub-{subnum}_task-{runtype}_run-{runnum}_space-fsLR_den-91k_bold_timeseries.tsv"
    data = pd.read_csv(fname, sep="\t")
    return data

def read_metadata(subnum, dataloc="../tseries", runtype="pair", runnum=1):
    fname = f"{dataloc}/sub-{subnum}_task-{runtype}_run-0{runnum}_events.tsv"
    dat = pd.read_csv(fname, sep="\t")
    return dat

def reducing_metadata(image_of_interest, subnum="01"):
    meta = read_metadata(subnum)
    img_rows = meta[meta['trial_type'] == image_of_interest]
    onset_column = img_rows["onset"].to_list()
    int_list = []
    for i in onset_column:
        TR = (i/1.5) + 3
        TR = int(math.ceil(TR))
        int_list.append(TR)
    return int_list

def pull_timeseries(imagepair,subnum="01"):
    int_list = reducing_metadata(imagepair)
    tseries = read_data(subnum)
    all_trows = []
    for i in int_list:
        trow = tseries.iloc[i]
        all_trows.append(trow)
    df = pd.concat(all_trows, axis=1, ignore_index=False)
    df = (df.transpose())
    display(df)
    
def main():
    A1 = "c47-34_29A"
    pull_timeseries(A1)

main()
    
    
    

,7Networks_LH_Vis_1,7Networks_LH_Vis_2,7Networks_LH_Vis_3,7Networks_LH_Vis_4,7Networks_LH_Vis_5,7Networks_LH_Vis_6,7Networks_LH_Vis_7,7Networks_LH_Vis_8,7Networks_LH_Vis_9,7Networks_LH_Vis_10,...,7Networks_RH_Default_pCunPCC_11,7Networks_RH_Default_pCunPCC_12,7Networks_RH_Default_pCunPCC_13,7Networks_RH_Default_pCunPCC_14,7Networks_RH_Default_pCunPCC_15,7Networks_RH_Default_pCunPCC_16,7Networks_RH_Default_pCunPCC_17,7Networks_RH_Default_pCunPCC_18,7Networks_RH_Cont_pCun_2,7Networks_RH_Cont_pCun_4
7,0.815905,-0.537616,-0.060012,-0.246175,-0.718148,-0.018797,-0.931194,-0.580945,0.252151,-0.442175,...,-0.909774,0.104488,-1.065234,1.591322,-0.514525,-0.197556,-0.242896,-0.162824,-0.367419,-1.111739
73,-0.365496,0.322350,2.607300,0.929786,1.364108,0.920943,0.946128,0.865568,0.428962,-0.059126,...,1.092743,0.260550,1.939656,0.304859,0.009392,0.782901,0.336966,0.707512,1.266025,0.262090
85,0.726244,-1.280059,-0.377916,-1.176859,2.370073,-0.903786,-0.231034,-1.293075,-0.553350,-0.467695,...,0.594231,-0.144258,0.158675,0.725684,-0.681592,0.511584,0.664075,0.859101,0.649817,1.277994
141,1.799365,0.605342,-0.376742,0.322946,1.358757,-0.246881,-0.950532,0.036591,-0.239200,0.096573,...,-0.823360,-0.391767,0.852461,-0.255617,-0.141772,0.088381,1.746465,0.476105,-0.433617,1.080029
195,-0.480739,0.239341,-0.473650,-0.195432,0.569910,0.095572,-0.786097,-0.385514,-0.796777,0.618817,...,1.191373,1.414873,0.434431,0.812845,0.203357,0.255649,-0.134269,-1.364178,-0.259150,1.158107


1. For a given image, reduce the meta dataframe to only rows that correspond to that image
2. Pull the onset column from these rows (should be left with 5 numbers)
3. Divide all onsets by 1.5 (gets it into TR units), then add 3 (for hemodynamic lag)
4. Round these (this may have to be conditional), to get just 5 integers
5. Use those numbers as indices into the timeseries dataframe - should be left with 5 rows of data.
6. Scale to function that does this for a given subject, runtype, run number and trial type.